<a href="https://colab.research.google.com/github/kkkkkkkm/Torch/blob/main/Resnet/torch_Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import  torch.nn as nn
import torch.utils.model_zoo 

In [52]:
import torch
print(torch.__version__)

1.10.0+cu111


In [53]:
__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152', 'resnext50_32x4d', 'resnext101_32x8d',
           'wide_resnet50_2', 'wide_resnet101_2']

model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-f37072fd.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-b627a593.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-0676ba61.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-63fe2227.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-394f9c45.pth',
    'resnext50_32x4d': 'https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth',
    'resnext101_32x8d': 'https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth',
    'wide_resnet50_2': 'https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth',
    'wide_resnet101_2': 'https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth',
}           

In [66]:
def conv3x3(in_planes: int, out_planes: int, stride: 1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1)


def conv1x1(in_planes: int, out_planes: int, stride: int = 1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

In [67]:
class BasicBlock(nn.Module):
  expansion = 1
  
  def __init__(self, inplanes, planes, stride = 1, downsample = None):
    super(BasicBlock, self).__init__()
    
    self.conv1 = conv3x3(inplanes, planes, stride)
    self.bn1 = nn.BatchNorm2d(planes)
    self.relu = nn.ReLU(inplace = True)
    self.conv2 = conv3x3(planes, planes)
    slef.bn2 = nn.BatchNorm2d(planes)
    self.downsample = downsample
    self.stride = stride

  def forward(self, x):
    indentity = x
    
    out = self.conv1(x)
    out = self.bn1(out)
    out = self.relu(out)

    out = self.conv2(out)
    out = self.bn2(out)

    if self.downsample is not None:
      identity = self.downsample(x)

    out += identity
    out = self.relu(out)

    return out

In [125]:
class Bottleneck(nn.Module):
  expansion = 4

  def __init__(self, inplanes, planes, stride =1, downsample = None): #Bottleneck(64,64,1)
    super(Bottleneck, self).__init__()

    self.conv1 = conv1x1(inplanes, planes) #conv1x1(64,64)
    self.bn1 = nn.BatchNorm2d(planes)
    self.conv2 = conv3x3(planes, planes, stride) #conv3x3(64,64)
    self.bn2 = nn.BatchNorm2d(planes)
    self.conv3 = conv1x1(planes, planes * self.expansion) #conv1x1(64,256)
    self.bn3 = nn.BatchNorm2d(planes * self.expansion)
    self.downsample = downsample
    self.stride = stride
    self.relu = nn.ReLU(inplace = True)


  def forward(self, x):
    identity = x
    
    out = self.conv1(x) # 1x1 stride = 1
    out = self.bn1(out)
    out = self.relu(out)

    out = self.conv2(out) # 3x3 stride = stride
    out = self.bn2(out)
    out = self.relu(out)

    out = self.conv3(out) # 1x1 stride = 1 
    out = self.bn3(out)

    if self.downsample is not None:   #조건문을 통해 identity 원본과 layer를 거친 결과를 맞춰 줌
        identity = self.downsample(x) # downsample 함수

    out += identity
    out = self.relu(out)

    return out


In [126]:
class resnet(nn.Module):
  
  #model = Resnet(Bottleneck,  [3,4,6,3], **kwargs) # Resnet 50

  def __init__(self, block, layers, num_classes = 100, zero_init_residual = False):
    super(resnet, self).__init__()
    self.inplanes = 64
    #input = (3x224x224)
    self.conv1 = nn.Conv2d(3,64, kernel_size =7, stride = 2, padding = 3, bias = False)
    #output = [64x112x112]
    
    self.bn1 = nn.BatchNorm2d(64)
    self.relu = nn.ReLU(inplace = True)

    #input = [64, 112, 112]
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride = 2, padding = 1)
    #output= [64, 56, 56]

    self.layer1 = self._make_layer(block, 64, layers[0])
    self.layer2 = self._make_layer(block, 128, layers[1], stride = 2)
    self.layer3 = self._make_layer(block, 256, layers[2], stride = 2)
    self.layer4 = self._make_layer(block, 512, layers[3], stride = 2)

    self.avgpool = nn.AdaptiveAvgPool2d((1,1))
    self.fc = nn.Linear(512 * block.expansion, num_classes)

    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
      elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)
      
    if zero_init_residual:
      for m in self.modules():
        if isinstance(m, Bottleneck):
          nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
        elif isinstance(m, BasicBlock):
          nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]


    #self.inplanes = 64
    #self.layer1 = self._make_layer(Bottleneck, 64, 3)

    #sle
  def _make_layer(self, block, planes, blocks, stride = 1):
      downsample = None
      
      if stride != 1 or self.inplanes != planes * block.expansion: #64 != 64*4(256) #stride 가 1이 아니면 이미지 크기의 감소 존재
        downsample = nn.Sequential(  #input downsampling 이미지의 크기와 채널 수를 맞춰줌
            conv1x1(self.inplanes, planes*block.expansion, stride), #conv1x1(64, 256, stride = 1)  
            nn.BatchNorm2d(planes * block.expansion), #batchNorm2d(256)  
        )
      layers = []
      layers.append(block(self.inplanes, planes, stride, downsample))
      # layers.append(Bottleneck(64, 64, 1, downsample))

      self.inplanes = planes*block.expansion #self.inplanes 64 * 4 = 256

      for _ in range(1, blocks):
      #for _ in range(1, blocks = 3)
        layers.append(block(self.inplanes, planes))

      return nn.Sequential(*layers)
      
      # *layers 
     #self.layers1 = [
        #Bottleneck(64, 64, 1, downsample),
        #Bottleneck(256, 64)
        #Bottleneck(256, 64)
       #]

  def forward(self, x: Tensor):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [127]:
def resnet50(pretrained= False, **kwargs):
  model = resnet(Bottleneck, [3,4,6,3], **kwargs)
  return model

In [128]:
res = resnet50()

In [130]:
res

resnet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256,

In [49]:
import torchvision.models.resnet as resnet
res = resnet.resnet50()

In [ ]:
res